In [1]:
import math
import numpy as np
import sys,os

In [2]:
poly_dir = "Polynomials_Saved_Datasets"
if not os.path.exists(poly_dir):
    os.makedirs(poly_dir)

In [3]:
def isprime(n) :
    if (n <= 1):
        return False
    if (n <= 3):
        return True
 
    # This is checked so that we can skip
    # middle five numbers in below loop
    if (n % 2 == 0 or n % 3 == 0):
        return False
 
    i = 5
    while(i * i <= n):
        if (n % i) == 0 or n % (i + 2) == 0:
            return False
        i = i + 6
 
    return True

# Inverse of a Distribution for Primes of the Form a^n + D*b^n, D>0

In [4]:
def inverse_distribution_primes_a_b_nth(D, n):
    delta_x=0.0001
    integral = 0
    i_lst = []
    counter = 0
    limit = 0.1
    i_ = 0
    while counter <10:
        for j in range(i_+1,10000):
            integral += delta_x*(1/ (1+D * (j*delta_x)**n)**(2/n)) #(nrt(1 + D * (j*delta_x) **n,n/2)))
            #print (integral, "\t", limit , i_lst)
            if integral>limit:
                i_ = j
                i_lst.append(j)
                break
        limit = limit + 0.1
        counter+=1
    return i_lst

# Distribution of prime numbers of the form a^n + D*b^n,D>0

In [5]:
def list_creator(n):
    List = []
    for i in range(n):
        List.append(0)
    return List

In [6]:
def distribution_primes_a_b(N, D, n, LIST):
    len_List = len(LIST)
    counter_list = list_creator(len_List)
    for a in range(1,N):
        for b in range(1,a):
            q = a**n + D*b**n
            M = (N**n)/(D+1)
            if q<M and isprime(q):
                for i in range(len(LIST)):
                    if i==0 and b/a <= LIST[0]/10000:
                        counter_list[0]+=1 
                    elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                        counter_list[i]+=1 
    return counter_list
    

# Distribution of prime numbers of the form a^n + D*b^n, D>0 in Modulo 2

In [14]:
def distribution_primes_a_b_mod2(N, D, n, LIST):
    len_List = len(LIST)
    odd_odd = list_creator(len_List)#an empty when a is odd and b is odd4
    odd_even = list_creator(len_List)#an empty when a is odd and b is even
    even_odd = list_creator(len_List)#an empty when a is even and b is odd
    for a in range(1,N):
        for b in range(1,a):
            if a%2 == 1 and b%2 == 1:
                q = a**n + D*b**n
                M = (N**n)/(D+1)
                if q<M and isprime(q):
                    for i in range(len(LIST)):
                        if i==0 and b/a <= LIST[0]/10000:
                            odd_odd[0]+=1 
                        elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                            odd_odd[i]+=1 
            elif a%2 == 1 and b%2 == 0:
                q = a**n + D*b**n
                M = (N**n)/(D+1)
                if q<M and isprime(q):
                    for i in range(len(LIST)):
                        if i==0 and b/a <= LIST[0]/10000:
                            odd_even[0]+=1 
                        elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                            odd_even[i]+=1
            elif D%2 == 1 and a%2 == 0 and b%2 == 1:
                q = a**n + D*b**n
                M = (N**n)/(D+1)
                if q<M and isprime(q):
                    for i in range(len(LIST)):
                        if i==0 and b/a <= LIST[0]/10000:
                            even_odd[0]+=1 
                        elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                            even_odd[i]+=1 
    if D%2 == 1:
        return even_odd + odd_even
    else:
        return odd_odd + odd_even

In [10]:
def deviation(dist_list):
    return max(dist_list)/min(dist_list)

In [11]:
def testing_d_pos(rangeN, incre, D, n):
    print(f"Primes of the form a^{n} + {D}b^{n}")
    print()
    print("Calculating the inverse distribution..")
    inverse_list = inverse_distribution_primes_a_b_nth(D, n)
    
    print ("Calculating the distribution of primes....")
    test_arr = np.zeros((rangeN//incre, len(inverse_list)),  dtype=np.int32)
    test_arrMod2 = np.zeros((rangeN//incre, len(inverse_list)),  dtype=np.int32)
    i = 0
    for N in range(incre,rangeN+incre, incre):
        test_arr[i] = distribution_primes_a_b(N, D, n, inverse_list)
        #print (f"N = {N}  ", str(distribution_primes_a_b(N, D, n, inverse_list)), deviation(distribution_primes_a_b(N, D, n, inverse_list)))
        i+=1
    
    print ("Calculating the distribution of primes in mod2....")
    columns = len(distribution_primes_a_b_mod2(100, D, n, inverse_list))
    test_arrMod2 = np.zeros((rangeN//incre, columns),  dtype=np.int32)
    j = 0
    for Nn in range(incre,rangeN+incre, incre):
        test_arrMod2[j] = distribution_primes_a_b_mod2(Nn, D, n, inverse_list)
        #print (f"N = {N}  ", str(distribution_primes_a_b(N, D, n, inverse_list)), deviation(distribution_primes_a_b(N, D, n, inverse_list)))
        j+=1
    print("Saving my arrays in npz file.....")
    np.savez("Polynomials_Saved_Datasets/"+str(D)+ "_power_"+ str(n) +"_"+ str(incre)+"_to_"+str(rangeN), inverse = np.array(inverse_list) ,distri = test_arr, distriMod2 = test_arrMod2)
    print("Done")

# Inverse of a Distribution for Primes of the Form a^n + D*b^n, D<0

In [20]:
def neg_inverse_distribution_primes_a_b_nth(D, n):
    delta_x=0.0001
    integral = 0
    i_lst = []
    counter = 0
    limit = 0.1
    i_ = 0
    while counter <6:
        for j in range(i_+1,10000):
            integral += 4*delta_x*(1/ (1+D * (j*delta_x)**n)**(2/n)) #(nrt(1 + D * (j*delta_x) **n,n/2)))
            #print (integral, "\t", limit , i_lst)
            if integral>limit:
                i_ = j
                i_lst.append(j)
                break
        limit = limit + 0.1
        counter+=1
    return i_lst

# Distribution of prime numbers of the form a^n + D*b^n, D<0

In [21]:
def neg_distribution_primes_a_b(N, D, n, LIST):
    len_List = len(LIST)
    counter_list = list_creator(len_List)
    for a in range(1,N):
        l = math.floor(a/((-D)**(1/n)))
        for b in range(1,l):
            q = a**n + D*b**n
            M = (N**n) - ((N**n)/(2**n*(-D)))
            if q<M and isprime(q):
                for i in range(len(LIST)):
                    if i==0 and b/a <= LIST[0]/10000:
                        counter_list[0]+=1 
                    elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                        counter_list[i]+=1 
    return counter_list

# Distribution of prime numbers of the form a^n + D*b^n, D<0 Modulo 2

In [22]:
def neg_distribution_primes_a_b_mod2(N, D, n, LIST):
    len_List = len(LIST)
    odd_odd = list_creator(len_List)#an empty when a is odd and b is odd4
    odd_even = list_creator(len_List)#an empty when a is odd and b is even
    even_odd = list_creator(len_List)#an empty when a is even and b is odd
    for a in range(1,N):
        l = math.floor(a/((-D)**(1/n)))
        for b in range(1,l):
            if a%2 == 1 and b%2 == 1:
                q = a**n + D*b**n
                M = (N**n) - ((N**n)/(2**n*(-D)))
                if q<M and isprime(q):
                    for i in range(len(LIST)):
                        if i==0 and b/a <= LIST[0]/10000:
                            odd_odd[0]+=1 
                        elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                            odd_odd[i]+=1 
            elif a%2 == 1 and b%2 == 0:
                q = a**n + D*b**n
                M = (N**n) - ((N**n)/(2**n*(-D)))
                if q<M and isprime(q):
                    for i in range(len(LIST)):
                        if i==0 and b/a <= LIST[0]/10000:
                            odd_even[0]+=1 
                        elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                            odd_even[i]+=1
            elif D%2 == 1 and a%2 == 0 and b%2 == 1:
                q = a**n + D*b**n
                M = (N**n) - ((N**n)/(2**n*(-D)))
                if q<M and isprime(q):
                    for i in range(len(LIST)):
                        if i==0 and b/a <= LIST[0]/10000:
                            even_odd[0]+=1 
                        elif i!=0 and b/a <= LIST[i]/10000 and b/a > LIST[i-1]/10000:
                            even_odd[i]+=1 
    if D%2 == 1:
        return even_odd + odd_even
    else:
        return odd_odd + odd_even

In [27]:
def testing_d_neg(rangeN, incre, D, n):
    print(f"Primes of the form a^{n} + {D}b^{n}")
    print()
    print("Calculating the negative inverse distribution..")
    neg_inverse_list = neg_inverse_distribution_primes_a_b_nth(D, n)
    
    print ("Calculating the distribution of primes D<0....")
    test_arr = np.zeros((rangeN//incre, len(neg_inverse_list)),  dtype=np.int32)
    i = 0
    for N in range(incre,rangeN+incre, incre):
        test_arr[i] = neg_distribution_primes_a_b(N, D, n, neg_inverse_list)
        #print (f"N = {N}  ", str(distribution_primes_a_b(N, D, n, inverse_list)), deviation(distribution_primes_a_b(N, D, n, inverse_list)))
        i+=1
    
    print ("Calculating the distribution of primes in mod2 D<0....")
    columns = len(distribution_primes_a_b_mod2(100, D, n, neg_inverse_list))
    test_arrMod2 = np.zeros((rangeN//incre, columns),  dtype=np.int32)
    j = 0
    for Nn in range(incre,rangeN+incre, incre):
        test_arrMod2[j] = neg_distribution_primes_a_b_mod2(Nn, D, n, neg_inverse_list)
        #print (f"N = {N}  ", str(distribution_primes_a_b(N, D, n, inverse_list)), deviation(distribution_primes_a_b(N, D, n, inverse_list)))
        j+=1
    print("Saving my arrays in npz file.....")
    np.savez("Polynomials_Saved_Datasets/"+"neg_"+str(abs(D))+ "_power_"+ str(n) +"_"+ str(incre)+"_to_"+str(rangeN), inverse = np.array(neg_inverse_list) ,distri = test_arr, distriMod2 = test_arrMod2)
    print("Done")

# Testing: Inverse Distruntion for primes a^2 + b^2

In [12]:
testing_d_pos(4000, 500, 1, 2)


Primes of the form a^2 + 1b^2

Calculating the inverse distribution..
Calculating the distribution of primes....
Calculating the distribution of primes in mod2....
Saving my arrays in npz file.....
Done


## Testing: Inverse Distruntion for primes a^2 + 2b^2

In [16]:
testing_d_pos(4000, 500, 2, 2)

Primes of the form a^2 + 2b^2

Calculating the inverse distribution..
Calculating the distribution of primes....
Calculating the distribution of primes in mod2....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^2 + 3b^2

In [17]:
testing_d_pos(4000, 500, 3, 2)

Primes of the form a^2 + 3b^2

Calculating the inverse distribution..
Calculating the distribution of primes....
Calculating the distribution of primes in mod2....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^2 + 5b^2

In [18]:
testing_d_pos(4000, 500, 5, 2)

Primes of the form a^2 + 5b^2

Calculating the inverse distribution..
Calculating the distribution of primes....
Calculating the distribution of primes in mod2....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^2 + 6b^2

In [19]:
testing_d_pos(4000, 500, 6, 2)

Primes of the form a^2 + 6b^2

Calculating the inverse distribution..
Calculating the distribution of primes....
Calculating the distribution of primes in mod2....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distribution for primes a^2 - 3b^2

In [28]:
testing_d_neg(4000, 500, -3, 2)

Primes of the form a^2 + -3b^2

Calculating the negative inverse distribution..
Calculating the distribution of primes D<0....
Calculating the distribution of primes in mod2 D<0....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^2-2b^2

In [37]:
testing_d_neg(4000, 500, -2, 2)

Primes of the form a^2 + -2b^2

Calculating the negative inverse distribution..
Calculating the distribution of primes D<0....
Calculating the distribution of primes in mod2 D<0....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^2-5b^2

In [38]:
testing_d_neg(4000, 500, -5, 2)

Primes of the form a^2 + -5b^2

Calculating the negative inverse distribution..
Calculating the distribution of primes D<0....
Calculating the distribution of primes in mod2 D<0....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^3-2b^3

In [41]:
testing_d_neg(3000, 500, -2, 3)

Primes of the form a^3 + -2b^3

Calculating the negative inverse distribution..
Calculating the distribution of primes D<0....
Calculating the distribution of primes in mod2 D<0....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^3-5b^3

In [42]:
testing_d_neg(3000, 500, -5, 3)

Primes of the form a^3 + -5b^3

Calculating the negative inverse distribution..
Calculating the distribution of primes D<0....
Calculating the distribution of primes in mod2 D<0....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^3-6b^3

In [43]:
testing_d_neg(3000, 500, -6, 3)

Primes of the form a^3 + -6b^3

Calculating the negative inverse distribution..
Calculating the distribution of primes D<0....
Calculating the distribution of primes in mod2 D<0....
Saving my arrays in npz file.....
Done


# Testing: Inverse Distruntion for primes a^4-2b^4

In [48]:
testing_d_neg(2000, 500, -2, 4)

Primes of the form a^4 + -2b^4

Calculating the negative inverse distribution..
Calculating the distribution of primes D<0....
Calculating the distribution of primes in mod2 D<0....


KeyboardInterrupt: 

# Testing: Inverse Distruntion for primes a^4-3b^4

In [ ]:
testing_d_neg(2000, 500, -3, 4)

# Testing: Inverse Distruntion for primes a^4-5b^4

In [ ]:
testing_d_neg(2000, 500, -5, 4)

# Testing: Inverse Distruntion for primes a^4-6b^4

In [ ]:
testing_d_neg(2000, 500, -6, 4)

# Testing: Inverse Distruntion for primes a^4-11b^4

In [ ]:
testing_d_neg(2000, 500, -11, 4)

# Testing: Inverse Distruntion for primes a^4 + 11b^4

# Testing: Inverse Distruntion for primes a^4 + b^4

# Testing: Inverse Distruntion for primes a^4 + 2b^4

# Testing: Inverse Distruntion for primes a^4 + 3b^4

# Testing: Inverse Distruntion for primes a^4 + 5b^4

# Testing: Inverse Distruntion for primes a^4 + 6b^4

# Testing: Inverse Distruntion for primes a^3 + 2b^3

# Testing: Inverse Distruntion for primes a^3 + 3b^3

# Testing: Inverse Distruntion for primes a^3 + 5b^3

# Testing: Inverse Distruntion for primes a^3 + 6b^3